[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pablo-sampaio/rl_facil/blob/main/capExtra/capX-crossentropy.ipynb)

# Método Cross-Entropy


## 1 - Configurações Iniciais

### Cria Diretório para Experimentos

In [ ]:
!mkdir results

### Configurações Dependentes do Sistema

Rode a célula abaixo, mesmo sem estar no Colab.

In [ ]:
import sys
from IPython.display import clear_output

if 'google.colab' in sys.modules:
    !pip install swig
    !pip install gymnasium[box2d]

    !git clone https://github.com/pablo-sampaio/rl_facil
    sys.path.append("/content/rl_facil")

    clear_output()
else:
    from os import path
    sys.path.append( path.dirname( path.dirname( path.abspath("__main__") ) ) )


### Imports Principais

Import algoritmos, ambientes e outros módulos.

In [ ]:
import gymnasium as gym
import numpy as np

from util.experiments import repeated_exec
from util.plot import plot_result, plot_multiple_results
from util.notebook import display_videos_from_path

from capExtra.models_torch import test_policy
from capExtra.crossentropy_method import run_crossentropy_method, PolicyModelCrossentropy

## 2 - Rodando o Método Cross-Entropy (v1)

In [ ]:
#ENV_NAME, rmax = "CartPole-v1", 500
ENV_NAME, rmax = "Acrobot-v1", 0
#ENV_NAME, rmax = "LunarLander-v2", 300
#ENV_NAME, rmax = "MountainCar-v0", 0  # resultados ruins

env = gym.make(ENV_NAME)

EPISODES   = 500    # total de episódios
BATCH_SIZE = 10      # quantidade de episódios executados por época de treinamento
PROPORTION = 0.2    # percentual dos episódios (do batch) que serão selecionados

policy_model = PolicyModelCrossentropy(env.observation_space.shape[0], [512], env.action_space.n, lr=0.005)
returns, policy1 = run_crossentropy_method(env, EPISODES, BATCH_SIZE, PROPORTION, verbose=True)
clear_output()

print("Últimos episódios do treinamento: media =", np.mean(returns[-20:]), ", desvio padrao =", np.std(returns[-20:]))


In [ ]:
# Exibe um gráfico episódios x retornos (não descontados)
plot_result(returns, rmax)

In [ ]:
# Roda alguns episódigos com o modelo e salva os vídeos em arquivos
env1 = gym.make(ENV_NAME, render_mode="rgb_array")
video_env=gym.wrappers.RecordVideo(env1, "./videos", episode_trigger=(lambda ep : True), video_length=3_000)
test_policy(video_env, policy1, False, 3)
video_env.close()

In [ ]:
display_videos_from_path('./videos')

## 3 - Experimentos


In [ ]:
NUM_EPISODES = 500
RUNS         = 5
AUTO_LOAD    = True

### 3.1 Variando Parâmetros do Algoritmo

Abaixo, mostramos experimentos variando os dois parâmetros mais diretos do método:
- o tamanho de batch (de episódios)
- a proporção dos episódios selecionados

***Atenção***: de forma geral, os resultados variam muito, porque dependem da inicialização da rede neural, que é aleatória. Para resultados mais confiáveis, use um número mais alto de repetições do que estamos usando abaixo. Tente com algumas *dezenas*.

In [ ]:
env = gym.make(ENV_NAME)


results1 = []

for batch_size in [5, 20]:
    _results = []
    for proportion in [1/5.0, 2/5.0, 4/5.0]:
        _results.append( repeated_exec(RUNS, f"CrossEntropy ({batch_size}; {proportion:.2f})", run_crossentropy_method, env, NUM_EPISODES, batch_size, proportion, auto_load=AUTO_LOAD) )
    plot_multiple_results(_results, cumulative='no')
    results1.extend(_results)


In [ ]:
plot_multiple_results(results1, cumulative='no')

### 3.2 Variando Parâmetros da Rede Neural (da Política)

Você também pode variar todos os parâmetros da rede neural: quantidade de camadas, taxa de aprendizagem, etc.

In [ ]:
env = gym.make(ENV_NAME)

BATCH_SIZE   = 20
PROPORTION   = 0.4
AUTO_LOAD    = True

In [ ]:
results2 = []

for hidden_layers in [ [32], [256], [256,256] ]:
    policy_model = PolicyModelCrossentropy(env.observation_space.shape[0], hidden_layers, env.action_space.n, lr=0.005)
    results2.append( repeated_exec(RUNS, f"CrossEntropy (hlayers={hidden_layers})", run_crossentropy_method, env, NUM_EPISODES, BATCH_SIZE, PROPORTION, policy_model, auto_load=AUTO_LOAD) )
    clear_output()

plot_multiple_results(results2, cumulative='no', x_log_scale=False)